In [13]:
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [14]:
data_path = "depression_dataset/data"

# Dosya oku ve birleştir
control_files = [f for f in os.listdir(os.path.join(data_path, "control")) if f.endswith('.csv')]
condition_files = [f for f in os.listdir(os.path.join(data_path, "condition")) if f.endswith('.csv')]

# Kontrol ve condition verilerini ayrı listelere yükle
control_data = [pd.read_csv(os.path.join(data_path, "control", file)) for file in control_files]
condition_data = [pd.read_csv(os.path.join(data_path, "condition", file)) for file in condition_files]

In [15]:
# Her bir grup için hedef değişken
for df in control_data:
    df['target'] = 0

for df in condition_data:
    df['target'] = 1

In [16]:
# Verileri birleştir, tarih ve zaman tanımla
data = pd.concat(control_data + condition_data, ignore_index=True)
data = data.drop(columns=['timestamp', 'date'])
data

,activity,target
0,7,0
1,5,0
2,5,0
3,5,0
4,5,0
...,...,...
1571701,0,1
1571702,0,1
1571703,172,1
1571704,0,1


In [17]:
#Encoder
if 'activity' in data.columns:
    label_encoder = LabelEncoder()
    data['activity'] = label_encoder.fit_transform(data['activity'])

In [18]:
# Veri setini böl 
X = data.drop(columns=['target'])
y = data['target']

# Test, train, val
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

In [ ]:
# Model tanımla
models = {
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC()
}

# Modelleri eğitmi
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    print(f"{model_name} - Accuracy: {accuracy_score(y_val, y_pred):.4f}, Precision: {precision_score(y_val, y_pred):.4f}, Recall: {recall_score(y_val, y_pred):.4f}, F1-score: {f1_score(y_val, y_pred):.4f}")

Random Forest - Accuracy: 0.6628, Precision: 0.5909, Recall: 0.1282, F1-score: 0.2107


In [ ]:
# En iyi modeli seçip test seti üzerinde değerlendiriyoruz
best_model = RandomForestClassifier()
best_model.fit(X_train, y_train)
y_test_pred = best_model.predict(X_test)

In [ ]:
# Test seti üzerinde performans değerlendirmesi
print(f"Test Set - Accuracy: {accuracy_score(y_test, y_test_pred):.4f}, Precision: {precision_score(y_test, y_test_pred):.4f}, Recall: {recall_score(y_test, y_test_pred):.4f}, F1-score: {f1_score(y_test, y_test_pred):.4f}")